In [17]:
import os
# Get the directory of the file
file_path = r"C:\Users\osi0pr\Documents\Github\Masterarbeit\Centrality\distance.ipynb"
directory = os.path.dirname(file_path)

# Change the current working directory
os.chdir(directory)

Load data

In [18]:
#load data from csv file
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#load data from csv file
bench = pd.read_excel('data/amadeus_benchmark_V4.xlsx')
industry = pd.read_excel('data/locations_hydro_V10.xlsx')

industry

,Lat,Lon,Name,Full Address,Address,ZIP,City,Country,Source
0,4.929128,-52.300176,CEOG,"PEPINIERE GDI CAMPUS UNIVERSITAIRE, LIEU DIT T...","PEPINIERE GDI CAMPUS UNIVERSITAIRE, LIEU DIT T...",97300,CAYENNE,France,Amadeus
1,5.668733,-53.778404,Centrale Électrique de l’Ouest Guyanais (CEOG),NaN,NaN,NaN,French Guiana,France,Hydrogen Production and Consumption Projects |...
2,14.605086,-61.044059,OB SOLUTIONS CARAIBE SARL,"ZONE FRANCHE DE DILLON, 17 RUE GEORGES EUCHARI...","ZONE FRANCHE DE DILLON, 17 RUE GEORGES EUCHARIS",97200,FORT DE FRANCE,France,Amadeus
3,20.990585,-89.632295,SYDKRAFT HYDROGEN AB,"BOX 94, 201 20, Skåne, Schweden",BOX 94,201 20,MALMÖ,Sweden,Amadeus
4,23.730443,90.411897,MER SKOPJE AD,"MAKSIM GORKI 11/1-3, 1000, Скопски, Nordmazedo...",MAKSIM GORKI 11/1-3,1000,SKOPJE,North Macedonia,Amadeus
...,...,...,...,...,...,...,...,...,...
3963,68.446745,17.449377,TECO2030 POWER SYSTEMS AS,"TEKNOLOGIVEIEN 6, 8517, Narvik, Norwegen",TEKNOLOGIVEIEN 6,8517,NARVIK,Norway,Amadeus
3964,70.799045,29.150529,HAEOLUS,NaN,NaN,NaN,Berlevag,Norway,Hydrogen Production | European Hydrogen Observ...
3965,71.042765,27.845717,FINNMARK COD AS,"RAET 9, 9770, Gamvik, Norwegen",RAET 9,9770,MEHAMN,Norway,Amadeus
3966,41.173800,-8.658730,CATIM - Portuguese Technological Center for th...,"Rua dos Plátanos, 197, 4100-414 - Porto",NaN,NaN,NaN,Portugal,Hydrogen Europe Research


Load code necessary

In [44]:
import math
from math import exp
from haversine import haversine, Unit

def calcDi(industry, dist, fun):  # Modified to accept dist variable
    thresh = 1  # min distance
    Di = [0] * len(industry)
    for i in range(len(Di)):
        print(i)  
        for z in range(i, len(Di)):  
            if dist[z] < thresh:
                dist[z] = thresh
            if fun == "invert":
                val = 1 / (dist[z] * len(Di))
            elif fun == "expo":
                val = exp(-0.05 * dist[z]) / len(Di)
            Di[i] += val
            Di[z] += val
    return Di

def bilatDistance(industry):
    bilateral_distances = []
    for i in range(len(industry)):
        print("Progress: ", i / len(industry) * 100, "%")
        for j in range(i + 1, len(industry)):
            distance = haversine((industry['Lat'][i], industry['Lon'][i]), (industry['Lat'][j], industry['Lon'][j]), unit=Unit.KILOMETERS)
            bilateral_distances.append(distance)
    return bilateral_distances


def convertXY(latitude, longitude):
    """
    Convert latitude and longitude coordinates to Cartesian coordinates for Earth.
    
    Args:
        latitude (float): Latitude coordinate in degrees.
        longitude (float): Longitude coordinate in degrees.
        
    Returns:
        tuple: Cartesian coordinates (x, y, z).
    """
    EARTH_RADIUS = 6371
    lat_radians = math.radians(latitude)
    long_radians = math.radians(longitude)
    
    x = EARTH_RADIUS * math.cos(lat_radians) * math.cos(long_radians)
    y = EARTH_RADIUS * math.cos(lat_radians) * math.sin(long_radians)
    
    return x, y

actually work with my data

In [29]:
# Apply the function to each row of the DataFrame to convert lon/lat to x/y
industry[['x', 'y']] = industry.apply(lambda row: pd.Series(convertXY(row['Lat'], row['Lon'])), axis=1)
bench[['x', 'y']] = bench.apply(lambda row: pd.Series(convertXY(row['Lat'], row['Lon'])), axis=1)

# Now, industry DataFrame contains 'x' and 'y' columns with Cartesian coordinates
#show me the row 3879 of industry
industry.iloc[3880]



Lat                                                60.39365
Lon                                                5.327617
Name                                     HYDROGEN VIKING AS
Full Address    VESTRE SKOSTREDET 2, 5017, Bergen, Norwegen
Address                                 VESTRE SKOSTREDET 2
ZIP                                                    5017
City                                                 BERGEN
Country                                              Norway
Source                                              Amadeus
x                                               3133.920453
y                                                292.248657
Name: 3880, dtype: object

Calculate bilat distances

In [45]:
distances_in = bilatDistance(industry)
#distances_be = bilatDistance(bench)


CEOG 0
Centrale Électrique de l’Ouest Guyanais (CEOG) 1
OB SOLUTIONS CARAIBE SARL 2
SYDKRAFT HYDROGEN AB 3
MER SKOPJE AD 4
ISGACAN, SOCIEDAD LIMITADA. 5
Plocan - Oceanic Platform of the Canary Islands 6
WATER2KW, SOCIEDAD LIMITADA. 7
CANALUZ INFINITA, SOCIEDAD LIMITADA. 8
QUIMICAS MORALES SL 9
Grupo DISA 10
HIDROGENO RENOVABLE SANTA CRUZ SL. 11
Tenerife Biomca Química 12
PIPELIFE HAFAB AKTIEBOLAG 13
NOURYON PULP AND PERFORMANCE CHEMICALS AB 14
SOL HELLAS Α.Ε. 15
IRESEN - Institute Research Energy Solar And Energy Nouvelles 16
Helbio 17
HYDVILLE SYSTEMS LTD 18
Evos Algeciras SAU 19
ITM Testing Spain S.L. 20
(SOLMAR) 21
Hydrogreen Energy 22
San Roque/Algeciras CEPSA 23
ANSASOL S.L. 24
AMT VENTUS OFFSHORE SL. 25
SOLUTIO INGENIERIA 26
TCI GECOMP SL 27
OXIGENO ALMERIA SOCIEDAD LIMITADA 28
Gela Praoil 29
MYTILINEOS 30
Priolo Versalis 31
Priolo Air Liquide 32
Priolo ISAB SpA (sud) 33
Melilli ISAB SpA (nord) 34
GENERAL COMPANY H2 GREEN S.L. 35
Centro de Investigación en Tecnología, Energía y S

KeyboardInterrupt: 

Calculate the Di Value

In [ ]:
Di_values = calcDi(industry, distances_in, fun="invert")



Now that I have both the Lon, Lat, X and Y coordinates - I can play around with the code

In [ ]:
from haversine import haversine, Unit

def calculate_bilateral_distances(industry):
    bilateral_distances = {}
    for i in range(len(industry)):
        for j in range(i + 1, len(industry)):
            distance = haversine((industry['Lat'][i], industry['Lon'][i]), (industry['Lat'][j], industry['Lon'][j]), unit=Unit.KILOMETERS)
            bilateral_distances[(industry['Name'][i], industry['Name'][j])] = distance
    return bilateral_distances

# Example usage:
bilateral_distances = calculate_bilateral_distances(industry)
print(bilateral_distances)